In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
import numpy as np
import random

c:\Users\Bryan Sukidi\Projects\interview_package_murge_ambiguity_project\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
np.random.seed(12)
torch.manual_seed(12)
random.seed(12)

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model_name = "mistralai/Mistral-7B-v0.1"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto", load_in_4bit=True)

c:\Users\Bryan Sukidi\Projects\interview_package_murge_ambiguity_project\venv\lib\site-packages\huggingface_hub\file_download.py:1142: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
Loading checkpoint shards: 100%|██████████| 2/2 [00:40<00:00, 20.20s/it]
c:\Users\Bryan Sukidi\Projects\interview_package_murge_ambiguity_project\venv\lib\site-packages\huggingface_hub\file_download.py:1142: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [62]:
prompt = """You are an intelligent robot. Your goal is to drop a knife in the living room. Knife is in the kitchen. You can navigate the environment, pick up items, and drop them.

You are in the living room. 
You see: couch, television, book.
You have the following items in your inventory: .
Valid actions: pick up couch, pick up television, pick up book, go to kitchen, go to bathroom, go to bedroom

Action: """

valid_actions = [
    "pick up couch",
    "pick up television",
    "pick up book",
    "go to kitchen",
    "go to bathroom",
    "go to bedroom",
]

prompt = """You are an intelligent robot. Your goal is to drop a knife in the living room. You can navigate the environment, pick up items, and drop them.

You are in the kitchen.
You see: fork, spoon.
You have the following items in your inventory: knife.
Valid actions: go to bathroom, go to bedroom, go to living room    

Action:"""

valid_actions = [
    "go to bathroom",
    "go to bedroom",
    "go to living room",
]

In [139]:
# Define the query and options
query = prompt
options = valid_actions

# Store results
results = []

# Calculate probability for each option
for option in options:
    # Encode sequences
    prompt_action_str = f"{query} {option}"
    prompt_action_tokens = tokenizer.encode(prompt_action_str, return_tensors="pt")
    
    prompt_tokens = tokenizer.encode(query, return_tensors="pt")
    prompt_length = prompt_tokens.shape[1]
    
    with torch.no_grad():
        outputs = model(prompt_action_tokens)
        logits = outputs.logits
    
    # get logits and logprobs
    logits = logits[:, prompt_length-1:-1, :]
    log_probs = torch.log_softmax(logits, dim=-1)

    # log_probs shape is [1, n, vocab_size]
    # action_tokens shape is [1, n]
    action_tokens = prompt_action_tokens[:, prompt_length:]
    token_log_probs = log_probs[0, torch.arange(action_tokens.shape[1]), action_tokens[0]]
    
    sequence_log_prob = token_log_probs.sum().item()
    results.append((option, sequence_log_prob))

# Sort and print results
results.sort(key=lambda x: x[1], reverse=True)
print("\nRanked completion probabilities (log scale):")
for option, log_prob in results:
    print(f'"{option}": {log_prob:.4f}')


Ranked completion probabilities (log scale):
"go to living room": -0.9896
"go to bathroom": -2.3422
"go to bedroom": -2.4047


In [138]:
type(results)

list

In [135]:
results[0][0]

'go to living room'

In [89]:
log_probs[0, ]

torch.Size([1, 3, 32000])

In [49]:
prompt = """You are an intelligent robot. Your goal is to drop a knife in the living room. Knife is in the kitchen. You can navigate the environment, pick up items, and drop them.

You are in the living room. 
You see: couch, television, book.
You have the following items in your inventory: .
Valid actions: pick up couch, pick up television, pick up book, go to kitchen, go to bathroom, go to bedroom

Action: """

valid_actions = [
    "pick up couch",
    "pick up television",
    "pick up book",
    "go to kitchen",
    "go to bathroom",
    "go to bedroom",
]

scores = {}

print(prompt)
print("------------------------------------------------------------")

for action in valid_actions:
    query = prompt + action

    inputs = tokenizer(query, return_tensors="pt").to(device)

    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits

    # applied log softmax to get log probabilities (log_probs)
    log_probs = torch.log_softmax(logits, dim=-1)

    # only interested in the action log probabilities, so we select the last few logits (using action token)
    action_tokens = tokenizer.encode(action, add_special_tokens=False)
    num_tokens = len(action_tokens)
    print(f"Action tokens: {num_tokens}")

    log_probs = log_probs[0, -num_tokens:, :]
    total_token_score = 0
    for i, token in enumerate(action_tokens):
        print(f"'{tokenizer.decode(token)}': {token_score}")
        token_score = log_probs[i][token].item()
        total_token_score += token_score
    
    print(f"action '{action}': {total_token_score}")
    print()
    

    scores[action] = total_token_score

print("------------------------------------------------------------")
sorted_scores = sorted(scores.items(), key=lambda item: item[1], reverse=True)
print("Sorted Scores:")
for action, score in sorted_scores:
    print(f"{action}: {score}")
print("------------------------------------------------------------")



    # print(f"Log probabilities for action '{action}': {log_probs.shape}")

You are an intelligent robot. Your goal is to drop a knife in the living room. Knife is in the kitchen. You can navigate the environment, pick up items, and drop them.

You are in the living room. 
You see: couch, television, book.
You have the following items in your inventory: .
Valid actions: pick up couch, pick up television, pick up book, go to kitchen, go to bathroom, go to bedroom

Action: go to 
------------------------------------------------------------
'kitchen': -11.349634170532227
action 'kitchen': -11.349634170532227

'bedroom': -9.390345573425293
action 'bedroom': -9.390345573425293

'bathroom': -10.493440628051758
action 'bathroom': -10.493440628051758

------------------------------------------------------------
Sorted Scores:
bedroom: -9.390345573425293
bathroom: -10.493440628051758
kitchen: -11.349634170532227
------------------------------------------------------------


In [29]:
# with torch.no_grad():
#     prompt = """You are an intelligent robot. Your goal is to drop a knife in the living room. Knife is in the kitchen. You can navigate the environment, pick up items, and drop them.

#     You are in the living room. 
#     You see: couch, television, book.
#     You have the following items in your inventory: .

#     Valid actions include:
#     pick up couch
#     pick up television
#     pick up book
#     go to living room
#     go to kitchen

#     Q: What action should you take that is the most relevant to your goal?
#     A: """

#     actions = [
#     "pick up couch",
#     "pick up television",
#     "pick up book",
#     "go to living room",
#     "go to kitchen"
#     ]

#     # Tokenize the input prompt
#     inputs = tokenizer(prompt, return_tensors="pt").to(device)

#     # Obtain logits
#     outputs = model(**inputs)
#     logits = outputs.logits

#     # Obtain log probabilities by applying log softmax to logits
#     log_probs = torch.log_softmax(logits, dim=1)

In [ ]:
log_probs.shape

In [115]:
prompt = """You are an intelligent robot. Your goal is to drop a knife in the living room. You can navigate the environment, pick up items, and drop them.

You are in the kitchen.
You see: knife, fork, spoon.
You have the following items in your inventory: .
Valid actions: pick up knife, pick up fork, pick up spoon

Action: """

valid_actions = [
    "pick up knife",
    "pick up fork",
    "pick up spoon",
]

In [116]:
# Initialize variables for tracking the best action
best_action = None
best_log_sum = float('-inf')  # Initialize to negative infinity

# Iterate over valid actions
for action in valid_actions:
    # Prepare the input sequence
    input_tokens = tokenizer.encode(prompt + action, add_special_tokens=False, return_tensors="pt").to(device)

    # Initialize log sum for this action
    log_sum = 0
    
    # Tokenize the action to get individual tokens
    action_tokens = tokenizer.encode(action, add_special_tokens=False, return_tensors="pt").to(device)
    for i in range(action_tokens.shape[1]):
        # Predict with the given model
        with torch.no_grad():
            outputs = model(input_tokens)
            logit_predictions = outputs.logits

        # Extract the log probability of the most recently added token
        last_token_logit = logit_predictions[0, -1, :]
        last_token_log_probs = torch.nn.functional.log_softmax(last_token_logit, dim=-1)
        log_token_prob = last_token_log_probs[action_tokens[0, i]].item()
        log_sum += log_token_prob

        # Incrementally add an output token to the current sequence
        last_token = action_tokens[:, i:i+1]
        input_tokens = torch.cat([input_tokens, last_token], dim=1)

    print(f"{action} - {log_sum}")
    # Check if the current action is the best one so far
    if log_sum > best_log_sum:
        best_log_sum = log_sum
        best_action = action

# Output the best action
print(f"Best Action: {best_action}")

pick up knife - -10.764559537172318
pick up fork - -10.66055477783084
pick up spoon - -10.227042868733406
Best Action: pick up spoon


In [ ]:
def get_action_probability(prompt, action):
    # Tokenize the prompt + action
    inputs = tokenizer(prompt, return_tensors="pt")
    action_tokens = tokenizer(action, return_tensors="pt")["input_ids"]

    # Get logits from the model for the prompt
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits

    # Softmax over the logits to get the probability distribution
    probs = torch.nn.functional.softmax(logits, dim=-1)

    # Extract probabilities for each token in the action
    action_prob = 1.0
    for i, token_id in enumerate(action_tokens[0]):
        action_prob *= probs[0, -(len(action_tokens[0]) - i), token_id].item()  # Prob for each token

    return action_prob


scores = []
for action in actions:
    score = get_action_probability(prompt, action)
    print(f"{action} | {score}")
    scores.append(score)

index = scores.index(max(scores))
print(f"Best Action: {actions[index]}")



In [58]:





# Initialize a dictionary to store scores
action_scores = {}

# Score each valid action
for action in valid_actions:
    # Create input for the model
    input_text = f"{prompt} {action}"
    input_tokens = tokenizer.encode(input_text, return_tensors="pt").to(device)

    # Get model outputs
    with torch.no_grad():
        outputs = model(input_tokens)
        logits = outputs.logits

    # Get the log probabilities of the last token
    last_token_logits = logits[0, -1, :]
    last_token_log_probs = torch.nn.functional.log_softmax(last_token_logits, dim=-1)

    # Get the token ID of the last token in the action
    action_tokens = tokenizer.encode(action, add_special_tokens=False)
    action_log_prob = sum(last_token_log_probs[token].item() for token in action_tokens)

    # Store the total log probability for the action
    action_scores[action] = action_log_prob

# Output the action scores
for action, score in action_scores.items():
    print(f"Action: {action}, Log Probability Score: {score}")


Action: pick up couch, Log Probability Score: -29.797097206115723
Action: pick up television, Log Probability Score: -30.571372032165527
Action: pick up book, Log Probability Score: -29.78604030609131
Action: go to kitchen, Log Probability Score: -27.17565393447876
Action: go to bedroom, Log Probability Score: -25.06926918029785
Action: go to bathroom, Log Probability Score: -25.485308170318604


In [166]:

def score_action(prompt: str, action: str):


    # Tokenize prompt + action, as well as action
    input_tokens = tokenizer(prompt + action, return_tensors="pt")
    action_tokens = tokenizer(action, return_tensors="pt")["input_ids"]

    # Compute logits
    with torch.no_grad():
        outputs = model(**input_tokens)
        logits = outputs.logits

    # Compute probability distribution
    probs = torch.nn.functional.log_softmax(logits, dim=-1)
    action_score = 0.0
    
    print(f"action = {action}")
    print(f"action token = {action_tokens[0]}")
    for idx, action_token_id in enumerate(action_tokens[0]):
        action_token_position = input_tokens["input_ids"].shape[1] - action_tokens.shape[1] + idx 
        # look up the probability
        print(f"action token position = {action_token_position}")
        action_probability = probs[0, action_token_position, action_token_id].item()
        print(f"action probability = {action_probability}")
        action_score += action_probability
        
    return action_score

In [162]:
prompt

'You are an intelligent robot. Your goal is to drop a knife in the living room. Knife is in the kitchen. You can navigate the environment, pick up items, and drop them.\n\nYou are in the living room. \nYou see: couch, television, book.\nYou have the following items in your inventory: .\nValid actions: pick up couch, pick up television, pick up book, go to kitchen, go to bathroom, go to bedroom\n\nThe knife is in the kitchen.\n\nAction: '

In [164]:
actions

['pick up couch',
 'pick up television',
 'pick up book',
 'go to kitchen',
 'go to bedroom',
 'go to bathroom']

In [167]:
for action in actions:
    score_action(prompt, action)

action = pick up couch
action token = tensor([    1,  3088,   582, 18602])
action token position = 110
action probability = -23.85727310180664
action token position = 111
action probability = -12.847025871276855
action token position = 112
action probability = -9.812251091003418
action token position = 113
action probability = -12.791942596435547
action = pick up television
action token = tensor([   1, 3088,  582, 8575])
action token position = 110
action probability = -23.85727310180664
action token position = 111
action probability = -12.847025871276855
action token position = 112
action probability = -9.812251091003418
action token position = 113
action probability = -11.886855125427246
action = pick up book
action token = tensor([   1, 3088,  582, 1820])
action token position = 110
action probability = -23.85727310180664
action token position = 111
action probability = -12.847025871276855
action token position = 112
action probability = -9.812251091003418
action token position = 11

In [158]:
print(repr(prompt))

def score_action(prompt: str, action: str):


    # Tokenize prompt + action, as well as action
    input_tokens = tokenizer(prompt + action, return_tensors="pt")
    action_tokens = tokenizer(action, return_tensors="pt")["input_ids"]

    untokenized_text = tokenizer.decode(input_tokens["input_ids"][0], skip_special_tokens=True)
    print("Untokenized text:", untokenized_text)
    print("-")
    # Compute logits
    with torch.no_grad():
        outputs = model(**input_tokens)
        logits = outputs.logits

    # Compute probability distribution
    probs = torch.nn.functional.log_softmax(logits, dim=-1)
    action_score = 1.0
    
    print(f"action = {action}")
    print(f"action token = {action_tokens[0]}")
    print(f"len input = {input_tokens.input_ids.shape[1]}")
    for idx, action_token_id in enumerate(action_tokens[0]):
        action_token_position = input_tokens["input_ids"].shape[1] - action_tokens.shape[1] + idx 
        # look up the probability
        print(f"action token position = {action_token_position}")
        action_probability = probs[0, action_token_position, action_token_id].item()
        print(f"action probability = {action_probability}")
        action_score *= action_probability
        
    return action_score
        

# Instruction, as used in the paper
scores = []
prompt = """You are an intelligent robot. Your goal is to drop a knife in the living room. Knife is in the kitchen. You can navigate the environment, pick up items, and drop them.

You are in the living room. 
You see: couch, television, book.
You have the following items in your inventory: .
Valid actions: pick up couch, pick up television, pick up book, go to kitchen, go to bathroom, go to bedroom

The knife is in the kitchen.

Action: """
for action in actions:
    # Probability that a skill is useful for instruction
    score = score_action(prompt, action)
    print(f"{action} | {score}")
    scores.append(score)

arg_max = actions[scores.index((scores))]
print(f"final action = {arg_max}")


'You are an intelligent robot. Your goal is to drop a knife in the living room. Knife is in the kitchen. You can navigate the environment, pick up items, and drop them.\n\nYou are in the living room. \nYou see: couch, television, book.\nYou have the following items in your inventory: .\nValid actions: pick up couch, pick up television, pick up book, go to kitchen, go to bathroom, go to bedroom\n\nThe knife is in the kitchen.\n\nAction: '
Untokenized text: You are an intelligent robot. Your goal is to drop a knife in the living room. Knife is in the kitchen. You can navigate the environment, pick up items, and drop them.

You are in the living room. 
You see: couch, television, book.
You have the following items in your inventory: .
Valid actions: pick up couch, pick up television, pick up book, go to kitchen, go to bathroom, go to bedroom

The knife is in the kitchen.

Action: pick up couch
-
action = pick up couch
action token = tensor([    1,  3088,   582, 18602])
len input = 114
act

ValueError: [1.748432676364631e-26, 4.322380443300403e-26, 1.431814865662497e-25, 5.812876914777729e-25, 1.1039648837740883e-23, 2.601255855800627e-24] is not in list

In [144]:
def score_action(prompt: str, action: str) -> float:

    # Tokenize prompt + action, as well as action
    input_tokens = tokenizer(prompt + action, return_tensors="pt")
    action_tokens = tokenizer(action, return_tensors="pt")["input_ids"]

    # Compute logits
    with torch.no_grad():
        outputs = model(**input_tokens)
        logits = outputs.logits

    # Compute probability distribution
    probs = torch.nn.functional.log_softmax(logits, dim=-1)
    action_score = 0

    # Calculate the score for the action
    for idx, action_token_id in enumerate(action_tokens[0]):
        action_token_position = input_tokens["input_ids"].shape[1] - action_tokens.shape[1] + idx
        action_probability = probs[0, action_token_position, action_token_id].item()
        action_score += action_probability

    return action_score

score_action(prompt, "go to k")

-47.34778928756714

In [156]:
score_action(prompt, "go to bathroom")

-46.714717388153076

In [ ]:
log_probs[0, prompt_len - 2]

In [ ]:
answer_tokens

In [39]:
def generate_constrained_response(prompt: str, valid_actions: list[str]) -> str:
    scores = []
    
    for action in valid_actions:
        # Combine prompt with action
        input_text = prompt + action
        
        # Tokenize
        input_tokens = tokenizer(input_text, return_tensors="pt")
        
        # Get model output
        with torch.no_grad():
            outputs = model(**input_tokens)
            logits = outputs.logits
        
        # Calculate probability
        probs = torch.nn.functional.softmax(logits, dim=-1)
        
        # Get the token IDs for the action
        action_token_ids = tokenizer(action)["input_ids"]
        
        # Access the last token's position for the action in the input
        action_prob = 1.0
        for idx, action_token_id in enumerate(action_token_ids):
            action_token_position = input_tokens["input_ids"].shape[1] - len(action_token_ids) + idx
            action_probability = probs[0, action_token_position, action_token_id].item()
            action_prob *= action_probability  # Combine probabilities
            
        scores.append(action_prob)
    
    # Select action with highest score
    best_action = valid_actions[scores.index(max(scores))]
    return best_action

# Example usage
generate_constrained_response(prompt, actions)


'go to bedroom'

In [46]:
untokenized_text = tokenizer.decode(tokens["input_ids"][0], skip_special_tokens=True)
print("Untokenized text:", untokenized_text)



NameError: name 'tokens' is not defined

In [129]:
combined_tokens = tokenizer.encode("He likes pizza with ugly women and toes", return_tensors='pt')


In [130]:
with torch.no_grad():
    outputs = model(combined_tokens)
    logits = outputs.logits  # shape: (batch_size, sequence_length, vocab_size)


In [131]:
# Get the token IDs for "pepperoni" and "and", "pizza"
token_ids = tokenizer.encode("ugly women and toes", return_tensors='pt')
# Get logits for the last tokens
relevant_logits = logits[0, -len(token_ids[0]):]  # Get the last len(token_ids) logits


In [132]:
probabilities = torch.nn.functional.softmax(relevant_logits, dim=-1)

In [138]:
probabilities.shape

torch.Size([5, 32000])

In [135]:
# Extract the indices for the tokens
indices = token_ids[0].tolist()

# Compute the joint probability
joint_probability = 1.0
for i in range(len(indices)):
    joint_probability += probabilities[i, indices[i]].item()


In [136]:
joint_probability

1.0004383147687157

In [122]:
joint_probability # joint_probability

1.107884229026856e-28